In [4]:
%gvim

<IPython.core.display.Javascript object>

Cell contents can now be edited via Gvim. From command mode use 'g' to open current cell contents in Gvim. After ':wq' from Gvim, use 'u' in command mode to update cell contents.


In [6]:
from scipy.optimize import linprog
from pulp import *
from functools import reduce

# linear optimisation with pulp

serves as a useful template for any LO problem

In [29]:
# create the model
model = LpProblem(name="example", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0, cat=LpInteger) 
y = LpVariable(name="y", lowBound=0, cat=LpInteger)

# constraints
model += (x <= 5)  # where x, y are numbers of projects
model += (y <= 5)
model += (x + 2*y <= 10)

# objectives
model += 2*x + 3*y # add the objective function last

print(model)

# solve
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}")

example:
MAXIMIZE
2*x + 3*y + 0
SUBJECT TO
_C1: x <= 5

_C2: y <= 5

_C3: x + 2 y <= 10

VARIABLES
0 <= x Integer
0 <= y Integer

status: 1, Optimal
objective: 17.0
x: 4.0
y: 3.0


# Converting non-linear problems to linear and solving

## i.e., step changes in the optimiation function

e.g.,
first 8 of product 1 make a 2.5 euro profit

profit = 2.5x + 3y - 0.5*max(0, x-8)

### non-linear equivilent:

profit = 2.5x + 3y - 0.5z

### Additional constraints
z >= x-8 (or rather x-z <= 8) , and z>= 0

In [32]:
# create the model
model = LpProblem(name="example", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0, cat=LpInteger) # ensure optimised to integer
y = LpVariable(name="y", lowBound=0, cat=LpInteger)
z = LpVariable(name="z", lowBound=0)

# constraints
model += (x <= 5)
model += (y <= 5)
model += (x + 2*y <= 10)
model += (z >= z-8)

# objectives
model += 2.5*x + 3*y - 0.5*z# add the objective function last

print(model)

# solve
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}")

example:
MAXIMIZE
2.5*x + 3*y + -0.5*z + 0.0
SUBJECT TO
_C1: x <= 5

_C2: y <= 5

_C3: x + 2 y <= 10

_C4: 0 z >= -8

VARIABLES
0 <= x Integer
0 <= y Integer
z Continuous

status: 1, Optimal
objective: 19.0
x: 4.0
y: 3.0
z: 0.0


# Project planning using L0 (with pulp)

In [41]:
# create the model
model = LpProblem(name="planning", sense=LpMinimize)

# task start times
a = LpVariable(name="a", lowBound=0, cat=LpInteger) 
b = LpVariable(name="b", lowBound=0, cat=LpInteger)
c = LpVariable(name="c", lowBound=0, cat=LpInteger)
d = LpVariable(name="d", lowBound=0, cat=LpInteger)
e = LpVariable(name="e", lowBound=0, cat=LpInteger)
f = LpVariable(name="f", lowBound=0, cat=LpInteger)
g = LpVariable(name="g", lowBound=0, cat=LpInteger)

# constraints
model += (b >= a + 2)
model += (c >= a + 2)
model += (d >= c + 2)
model += (e >= b + 3)
model += (e >= d + 1)
model += (g >= c + 2)
model += (g >= f + 3)
model += (e >= g + 2)

# object function (finish time to be minimised)
model += e + 2

print(model)

# solve
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}")

planning:
MINIMIZE
1*e + 2
SUBJECT TO
_C1: - a + b >= 2

_C2: - a + c >= 2

_C3: - c + d >= 2

_C4: - b + e >= 3

_C5: - d + e >= 1

_C6: - c + g >= 2

_C7: - f + g >= 3

_C8: e - g >= 2

VARIABLES
0 <= a Integer
0 <= b Integer
0 <= c Integer
0 <= d Integer
0 <= e Integer
0 <= f Integer
0 <= g Integer

status: 1, Optimal
objective: 8.0
a: 0.0
b: 2.0
c: 2.0
d: 4.0
e: 6.0
f: 0.0
g: 4.0


# knapsack problem

In [10]:
from pulp import *

# create the model
model = LpProblem(name="knapsack", sense=LpMaximize)

# Initialize the decision variables
items = []
for item in range(1, 8):
    items.append(LpVariable(name=f"item{item}", lowBound=0, cat=LpBinary))

# sizes
sizes = [3, 5, 4, 1.4, 3, 3, 1]

# reward
rewards = [60, 60, 40, 10, 20, 10, 3]

# constraints
model += sum([item * size for item, size in zip(items, sizes)]) <= 11

# objectives
model += sum([item * reward for item, reward in zip(items, rewards)])

print(model)

# solve
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}")




knapsack:
MAXIMIZE
60*item1 + 60*item2 + 40*item3 + 10*item4 + 20*item5 + 10*item6 + 3*item7 + 0
SUBJECT TO
_C1: 3 item1 + 5 item2 + 4 item3 + 1.4 item4 + 3 item5 + 3 item6 + item7 <= 11

VARIABLES
0 <= item1 <= 1 Integer
0 <= item2 <= 1 Integer
0 <= item3 <= 1 Integer
0 <= item4 <= 1 Integer
0 <= item5 <= 1 Integer
0 <= item6 <= 1 Integer
0 <= item7 <= 1 Integer

status: 1, Optimal
objective: 140.0
item1: 1.0
item2: 1.0
item3: 0.0
item4: 0.0
item5: 1.0
item6: 0.0
item7: 0.0


# Robust Regression (multivariate)

In [76]:
import numpy as np

# data points
features = np.array([[1],  
                     [5],
                     [8]])

targets = np.array([[3],
                    [4],
                    [0]])

# create the model
model = LpProblem(name="robust regression", sense=LpMinimize)

# initialise gradients, store in column vector
weights = []
num_features = features.shape[1]
for i, feature in enumerate(range(num_features)):
    weights.append(LpVariable(name=f"m{i}"))
weights = np.array(weights, ndmin=2) # [f x 1]

# initialise the y-intercept
c = LpVariable(name="c")  # scalar

# initialise the error variables, store as list of tuples
errors = []  # [(e1+, e1-),...]
for i, y in enumerate(targets):
    errors.append(
        (
            LpVariable(name=f"e{i}_pos", lowBound=0),
            LpVariable(name=f"e{i}_neg", lowBound=0) 
        )
    )

# add constraints to model
preds = np.matmul(features, weights) + c # [data x f][f x 1]
for i, pred in enumerate(preds):
    target = targets[i,0]
    pos, neg = errors[i]
    model += (pos + neg == target - pred)

# object function (finish time to be minimised)
model += (sum([e[0] + e[1] for e in errors]))

print(model)

# solve
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}")

robust_regression:
MINIMIZE
1*e0_neg + 1*e0_pos + 1*e1_neg + 1*e1_pos + 1*e2_neg + 1*e2_pos + 0
SUBJECT TO
_C1: c + e0_neg + e0_pos + m0 = 3

_C2: c + e1_neg + e1_pos + 5 m0 = 4

_C3: c + e2_neg + e2_pos + 8 m0 = 0

VARIABLES
c free Continuous
e0_neg Continuous
e0_pos Continuous
e1_neg Continuous
e1_pos Continuous
e2_neg Continuous
e2_pos Continuous
m0 free Continuous

status: 1, Optimal
objective: 2.7142857
c: 3.4285714
e0_neg: 0.0
e0_pos: 0.0
e1_neg: 2.7142857
e1_pos: 0.0
e2_neg: 0.0
e2_pos: 0.0
m0: -0.42857143


Note: check via direct matrix solution

(they won't be identical, but not highly dissimilar either)

In [97]:
from numpy.linalg import inv
from numpy import matmul as mm

X = np.copy(features)
X = np.append(X, np.ones(X.shape[0]).reshape(-1,1), axis=1)
print

w = mm(mm(inv(mm(X.T, X)), X.T), targets)
w

array([[-0.39189189],
       [ 4.16216216]])

# Multi-period inventory problem

balancing supply and stocks at time t, for given demand, such to minimise combined holding and order costs

Some more thought needed to apply this to real life scenario - to properly account for in/out/stock costs etc

E.g., in the example below
Supply@t is the supply ordered beween t-1 and t
demand@t is the demand between t-1 and t
stocks@t is the outstanding stock at t

where stock costs are simplified at stock@t x some charge

In [117]:
# initial stock
s0 = 8

# Data at times t
stock = [s0]

# add zeros for initial values
demand = [0, 5, 4, 8]  # per item demand
holding_costs = [0, 1, 1, 1]  # costs per item
order_costs = [0, 10, 13, 10] # costs per item
supply = [0]

# create the model
model = LpProblem(name="inventory", sense=LpMinimize)

# Initialize the decision variables

# how much stock and incoming supply at time t
for t in range(1, len(demand)+1):
    stock.append(LpVariable(name=f"stock{t}", lowBound=0, cat=LpInteger))
    supply.append(LpVariable(name=f"supply{t}", lowBound=0, cat=LpInteger))

# constraints:
# stock@t = stock@t-1 + supply@t - demand@t
for t in range(1, len(demand)):
    print(t)
    model += (stock[t] == stock[t-1] + supply[t] - demand[t])

# objectives
model += sum([s*c for s, c in zip(stock, holding_costs)]) + [s*c for s, c in zip(supply, order_costs)]

print(model)

# solve
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
# for name, constraint in model.constraints.items():
#     print(f"{name}: {constraint.value()}"]

1
2
3
inventory:
MINIMIZE
1*stock1 + 1*stock2 + 1*stock3 + 10*supply1 + 13*supply2 + 10*supply3 + 0
SUBJECT TO
_C1: stock1 - supply1 = 3

_C2: - stock1 + stock2 - supply2 = -4

_C3: - stock2 + stock3 - supply3 = -8

VARIABLES
0 <= stock1 Integer
0 <= stock2 Integer
0 <= stock3 Integer
0 <= supply1 Integer
0 <= supply2 Integer
0 <= supply3 Integer

status: 1, Optimal
objective: 94.0
stock1: 4.0
stock2: 0.0
stock3: 0.0
supply1: 1.0
supply2: 0.0
supply3: 8.0
